In [2]:
# !pip install fancyimpute
import numpy as np
import pandas as pd
from fancyimpute import IterativeImputer
from sklearn.preprocessing import StandardScaler

In [2]:
'''
pre-processes melanoma data

'''

'\npre-processes melanoma data\n\n'

In [3]:
# read in data

raw_data = pd.read_csv('../Melanoma/total_PD_1.csv')
raw_data.tail()

/var/folders/cn/ccmq_jg52yxblqm5cp00kr400000gn/T/ipykernel_53640/107892282.py:3: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('../Melanoma/total_PD_1.csv')


,Unnamed: 0,GSM2445716,GSM2445717,GSM2445718,GSM2445719,GSM2445720,GSM2445721,GSM2445722,GSM2445723,GSM2445724,...,SRR5088909,SRR5088911,SRR5088913,SRR5088914,SRR5088916,SRR5088920,SRR5088922,SRR5088924,SRR5088926,SRR5088929
56265,snoZ6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56266,snosnR66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56267,uc_338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.003664563208162,0.031978130771298,0.0857459707118245,0.04071053428664,0.0512846604842121,0.158733159882816,0.18278951659712,0.0657273971922831,0.0777414803400673,0.0
56268,yR211F11.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0355281824962792,0.0
56269,Response,N,N,N,N,N,N,N,N,N,...,N,R,N,N,N,N,N,R,N,R


In [5]:
# some re-formatting

# rename column

raw_data = raw_data.rename(columns={raw_data.columns[0]: 'patient'})
print(raw_data.shape)
#drop GSE931
raw_data_drop = raw_data.drop(raw_data.columns[raw_data.columns.str.contains("GSM24457")], axis=1)
print(raw_data_drop.shape)
# transpose so columns are genes and rows are patients
t_data = raw_data_drop.set_index('patient').T
print(t_data.shape)

# drop rows(patient) with resopnse = UNK

t_data = t_data[t_data["Response"] != "UNK"]
print(t_data.shape)
# split data

labels = t_data['Response']
clean_data = t_data.drop(columns = ['Response'])


(56270, 158)
(56270, 133)
(132, 56270)
(125, 56270)


In [6]:
# drop any genes with > 90% missing data

clean_data = clean_data.dropna(thresh = clean_data.shape[0]*0.9, axis=1)
clean_data.head()

patient,5S_rRNA,5_8S_rRNA,7SK,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,...,snoU2-30,snoU2_19,snoU83B,snoZ196,snoZ278,snoZ40,snoZ6,snosnR66,uc_338,yR211F11.2
SRR7344546,0.0,0.0,1.242231,0.131302,0.56498,0.036516,166.557144,1.199808,0.036012,0.0,...,3.13092825610652,3.28747466891185,0.0,0.0,0.0,0.0,0.0,0.0,0.416319623212903,0.0
SRR7344554,0.093389,0.0,1.564588,0.050331,0.721904,0.010498,221.367276,5.243057,0.165653,0.0,...,13.6818810287859,1.57521656581416,0.0,0.0,0.0,0.0,0.0,0.0,0.364929374639851,0.0
SRR7344556,0.118987,0.0,0.10454,0.0,0.0,0.008822,23.646357,1.623148,0.0116,0.0,...,4.23564377253204,0.264727735783252,0.0,0.0,0.0,0.0,0.0,0.0,0.487394747280299,0.0
SRR7344564,0.203915,0.0,0.02142,0.253573,3.659736,0.07603,140.576617,0.666165,0.039121,0.0,...,0.226744499164846,0.0,0.360729885034982,2.85341392207446,0.0,0.0,0.0,0.0,0.0747493692947006,0.0
SRR7344565,0.098599,0.0,0.037828,0.181927,2.509024,0.070055,101.541458,0.34527,0.901991,0.0,...,0.0,0.437981509864241,0.0,1.57476497928716,0.0,0.0,0.0,0.0,0.0583043344721167,0.0


In [7]:
# log2 transformation

# note that samples from the Melanoma dataset GSE931 are already log2 transformed

# add 1 to all values for log2 transformation

clean_data = clean_data.applymap(lambda x: x+1 if isinstance(x, (int, float)) else x)

# subset GSE931 samples from rest
# gse931 = clean_data[clean_data.index.str.contains('GSM24457')]

# to_transform = clean_data[~clean_data.index.str.contains("GSM24457")]

# log2 transform

log2_data = clean_data.applymap(lambda x: np.log2(x) if isinstance(x, (int, float)) else x)

# re-join data

# log2_data = pd.concat([gse931, log2_data], axis = 0)
# log2_data.head()


/var/folders/cn/ccmq_jg52yxblqm5cp00kr400000gn/T/ipykernel_53640/155180061.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clean_data = clean_data.applymap(lambda x: x+1 if isinstance(x, (int, float)) else x)
/var/folders/cn/ccmq_jg52yxblqm5cp00kr400000gn/T/ipykernel_53640/155180061.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  log2_data = clean_data.applymap(lambda x: np.log2(x) if isinstance(x, (int, float)) else x)


In [8]:
# missing data imputation

imputer = IterativeImputer()
imputed_data = imputer.fit_transform(log2_data)

imputed_data = pd.DataFrame(imputed_data, columns = log2_data.columns, index = log2_data.index)
imputed_data.head()

: 

In [17]:
# z-score transformation (for PCA)

scaler = StandardScaler()
scaled_data = scaler.fit_transform(imputed_data)

scaled_data = pd.DataFrame(scaled_data)
scaled_data.head()
print(scaled_data.shape)

(150, 717)


In [18]:
# export '/content/total_PD_1.csv'

# imputed log2 transformed data
# imputed_data.to_csv('imputed_data_dropUNK.csv')

# normalized version of above^ (for PCA)
scaled_data.to_csv('scaled_data_dropUNK_noname.csv')

# true labels (response y/n)
# labels.to_csv('labels_dropUNK.csv')